In [3]:
import sympy as sm
from IPython.display import display

In [4]:
sm.init_printing()

In [5]:
x: sm.Symbol
t: sm.Symbol
h: sm.Symbol
tau: sm.Symbol
u: sm.Function
lm: sm.Symbol
sg: sm.Symbol
x, t, h, tau, lm, sg = sm.symbols('x t h tau lambda sigma')
u = sm.Function('u')(x, t)

a1, a2, a3, a4 = sm.symbols('a_1 a_2 a_3 a_4')

In [6]:
def expansion(f: sm.Function, var: sm.Symbol, delta, n=4) -> sm.Function:
    return sum(delta ** i / sm.factorial(i) * f.diff(var, i) for i in range(n))


def expansion2(f: sm.Function, var1: sm.Symbol, var2: sm.Symbol, delta1, delta2):
    res = expansion(expansion(f, var1, delta1), var2, delta2).expand()
    return (res + sm.O(delta1 ** 4) + sm.O(delta2 ** 4) + sm.O(delta1 ** 2 * delta2 ** 2)
            + sm.O(delta1 * delta2 ** 3) + sm.O(delta1 ** 3 * delta2)).simplify().removeO()


In [7]:
assert expansion2(u, x, t, 0, 0) == u

![](img.png "Title")

$$
U_m^{n+1}=a_1U_{m-1}^n+a_2U_{m+1}^n+a_3U_m^{n-1}+a_4U_{m-2}^{n-1}
$$

In [16]:
expr = sm.Eq(expansion2(u, x, t, 0, tau), a1 * expansion2(u, x, t, -h, 0) + a2 * expansion2(u, x, t, h, 0) \
             + a3 * expansion2(u, x, t, 0, -tau) + a4 * expansion2(u, x, t, -2 * h, -tau))
display(expr)

     3                 2                                         ⎛       3    
 3  ∂              2  ∂                                          ⎜   3  ∂     
τ ⋅───(u(x, t))   τ ⋅───(u(x, t))                                ⎜  h ⋅───(u(x
     3                 2                                         ⎜       3    
   ∂t                ∂t               ∂                          ⎜     ∂x     
─────────────── + ─────────────── + τ⋅──(u(x, t)) + u(x, t) = a₁⋅⎜- ──────────
       6                 2            ∂t                         ⎝         6  
                                                                              

             2                                   ⎞      ⎛     3               
         2  ∂                                    ⎟      ⎜ 3  ∂              2 
, t))   h ⋅───(u(x, t))                          ⎟      ⎜h ⋅───(u(x, t))   h ⋅
             2                                   ⎟      ⎜     3               
           ∂x               ∂                    ⎟ 

$$
U_t=-\lambda U_x
\\
U_{tt}=-\lambda^2 U_{xx}
\\
U_{ttt}=-\lambda^3 U_{xxxx}
\\
U_{xt}=-\lambda U_{xx}
\\
U_{xtt}=\lambda^2 U_{xxx}
\\
U_{xxt}=-\lambda U_{xxx}
$$

In [17]:
expr = expr.subs(u.diff(t), -lm * u.diff(x)).simplify()
expr = expr.subs(u.diff(t, 2), lm ** 2 * u.diff(x, 2)).simplify()
expr = expr.subs(u.diff(t, 3), -lm ** 3 * u.diff(x, 3)).simplify()
expr = expr.subs(u.diff(x).diff(t), -lm * u.diff(x, 2)).simplify()
expr = expr.subs(u.diff(x).diff(t, 2), lm ** 2 * u.diff(x, 3)).simplify()
expr = expr.subs(u.diff(x, 2).diff(t), -lm * u.diff(x, 3)).simplify()
display(expr)

        3                    2                                           ⎛    
 3  3  ∂              2  2  ∂                                            ⎜ 3  
λ ⋅τ ⋅───(u(x, t))   λ ⋅τ ⋅───(u(x, t))                               a₁⋅⎜h ⋅─
        3                    2                                           ⎜    
      ∂x                   ∂x                 ∂                          ⎝   ∂
────────────────── - ────────────────── + λ⋅τ⋅──(u(x, t)) - u(x, t) = ────────
        6                    2                ∂x                              

 3                   2                                       ⎞      ⎛     3   
∂                2  ∂                 ∂                      ⎟      ⎜ 3  ∂    
──(u(x, t)) - 3⋅h ⋅───(u(x, t)) + 6⋅h⋅──(u(x, t)) - 6⋅u(x, t)⎟   a₂⋅⎜h ⋅───(u(
 3                   2                ∂x                     ⎟      ⎜     3   
x                  ∂x                                        ⎠      ⎝   ∂x    
───────────────────────────────────────────────────

In [18]:
expr2 = expr.lhs - expr.rhs
expr2 = sm.factor(expr2).collect(u)
display(expr2)

                                                                              
                                                                              
(a₁ + a₂ + a₃ + a₄ - 1)⋅u(x, t) + (-a₁⋅h + a₂⋅h + a₃⋅λ⋅τ - 2⋅a₄⋅h + a₄⋅λ⋅τ + λ
                                                                              
                                                                              

                  ⎛    2       2       2  2                              2  2 
    ∂             ⎜a₁⋅h    a₂⋅h    a₃⋅λ ⋅τ          2                a₄⋅λ ⋅τ  
⋅τ)⋅──(u(x, t)) + ⎜───── + ───── + ──────── + 2⋅a₄⋅h  - 2⋅a₄⋅h⋅λ⋅τ + ──────── 
    ∂x            ⎝  2       2        2                                 2     
                                                                              

   2  2⎞   2            ⎛      3       3       3  3         3                 
  λ ⋅τ ⎟  ∂             ⎜  a₁⋅h    a₂⋅h    a₃⋅λ ⋅τ    4⋅a₄⋅h          2       
- ─────⎟⋅───(u(x, t)) + ⎜- ───── + ───── + ───────

In [22]:
system = [
    sm.Eq(expr2.coeff(u), 0),
    sm.Eq(expr2.coeff(u.diff(x)) / h, 0).expand().subs(lm * tau / h, sg),
    sm.Eq(expr2.coeff(u.diff(x, 2)) * 2 / h ** 2, 0).expand().subs(lm * tau / h, sg),
    sm.Eq(expr2.coeff(u.diff(x, 3)) * 6 / h ** 3, 0).expand().subs(lm * tau / h, sg)
]

In [29]:
for i in system:
    display(sm.Eq(i.lhs.collect([a1, a2, a3, a4]).simplify(), 0))

a₁ + a₂ + a₃ + a₄ - 1 = 0

-a₁ + a₂ + a₃⋅σ + a₄⋅(σ - 2) + σ = 0

              2      ⎛ 2          ⎞    2    
a₁ + a₂ + a₃⋅σ  + a₄⋅⎝σ  - 4⋅σ + 4⎠ - σ  = 0

               3      ⎛ 3      2           ⎞    3    
-a₁ + a₂ + a₃⋅σ  + a₄⋅⎝σ  - 6⋅σ  + 12⋅σ - 8⎠ + σ  = 0

Первый порядок - первые два уравнения

In [32]:
system1 = sm.solve(system[:2], [a1, a2])

In [38]:
display(system1)

⎧    a₃⋅σ   a₃   a₄⋅σ   3⋅a₄   σ   1        a₃⋅σ   a₃   a₄⋅σ   a₄   σ   1⎫
⎨a₁: ──── - ── + ──── - ──── + ─ + ─, a₂: - ──── - ── - ──── + ── - ─ + ─⎬
⎩     2     2     2      2     2   2         2     2     2     2    2   2⎭

In [39]:
for i in [*system1.values(), a3, a4]:
    display(i >= 0)

a₃⋅σ   a₃   a₄⋅σ   3⋅a₄   σ   1    
──── - ── + ──── - ──── + ─ + ─ ≥ 0
 2     2     2      2     2   2    

  a₃⋅σ   a₃   a₄⋅σ   a₄   σ   1    
- ──── - ── - ──── + ── - ─ + ─ ≥ 0
   2     2     2     2    2   2    

a₃ ≥ 0

a₄ ≥ 0

Второй порядок - первые три уравнения

In [51]:
system2 = sm.solve(system[:3], [a1, a2, a3])
for i, j in system2.items():
    display(sm.Eq(i, j).simplify())

                    2    
     a₄⋅σ - 3⋅a₄ + σ  + σ
a₁ = ────────────────────
            σ + 1        

a₂ = -a₄ - σ

     -a₄⋅σ + 3⋅a₄ + σ + 1
a₃ = ────────────────────
            σ + 1        

Третий порядок - все уравнения

In [52]:
system3 = sm.solve(system, [a1, a2, a3, a4])
for i, j in system3.items():
    display(sm.Eq(i, j).simplify())

a₁ = 2⋅σ

     2⋅σ⋅(1 - σ)
a₂ = ───────────
        σ - 3   

a₃ + σ = 1

     σ⋅(σ + 1)
a₄ = ─────────
       σ - 3  